In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import requests
import json

from scipy.stats import entropy as entr
from numpy.linalg import norm
from pandas.tseries.offsets import MonthEnd
from time import time
from time import sleep

from pandas_datareader import data, wb

import statsmodels.formula.api as sm

import seaborn as sns
from  matplotlib import pyplot as plt
%matplotlib inline

sns.set_style("white")

In [2]:
def formatPythonList(string):
    return  [t.replace("{","").replace("}","").replace(",","").replace("*^","e") 
             for t in string.fields()[0]]


def mathematicaMaxEnt(list1, list2, norm=1):
    l1 = "List"+str(list1)
    l2 = "List"+str(list2)

    maxent = !./maxent.sh "$l1" "$l2" "$norm"
    maxent = formatPythonList(maxent)

    return np.array(map(np.float, maxent)).reshape(len(list1), len(list2))

In [3]:
def JSD(P, Q):
    p = np.array(P)
    q = np.array(Q)
    if len(p.shape) > 1:
        p = p.flatten()
        q = q.flatten()
    m = 0.5 * (p + q)
    return 0.5/np.log(2.) * (entr(p, m) + entr(q, m))


def KLD(P, Q):
    p = np.array(P)
    q = np.array(Q)
    if len(p.shape) > 1:
        p = p.flatten()
        q = q.flatten()
    return entr(p, q)


def S_KLD(X, Y, t0=None, t1=None):
    x = X.loc[t0:t1]
    y = Y.loc[t0:t1]
    
    x_bins = x.value_counts(True, True, True).sort_index().values.tolist()
    y_bins = y.value_counts(True, True, True).sort_index().values.tolist()
    
    maxent_distr = mathematicaMaxEnt(x_bins, y_bins) 
    emp_distr = pd.crosstab(x, y, normalize=True)
    
    return KLD(emp_distr, maxent_distr)


def rescaleRange(x):
    return 1 - np.exp(-2.*x)


def stringCSV(name, *arg):
    string = name
    if len(arg) > 0:
        string = string + "_"
        arg = [a for a in arg if a is not None]
        arg = map(str, arg)
        
    return string + "_".join(arg) + ".csv"

In [4]:
# Read returns, downloaded and preprocessed in corresponding file
df = pd.read_csv("data/M10_2005_2018_returns_filtered.csv", 
                 engine='c', index_col='time', parse_dates=True)

# Symbols to potentially analyze
symbols = ["EUR_AUD", "EUR_CAD", "EUR_CHF", "EUR_EUR", "EUR_GBP", "EUR_JPY", 
           "EUR_MXN", "EUR_NOK", "EUR_NZD", "EUR_SEK", "EUR_USD", "EUR_ZAR"]
curr = [sym[-3:] for sym in symbols]

In [714]:
# Class Currency Set. 
# Each instance comprises a set of select currencies during a certain time frame.
# Since part of the analysis is ranking currency returns, specifying the subset matters.
class currencySet(object):
    
    def __init__(self, df, symbols=None, t0=None, t1=None):
        
        if t0 == None:
            t0 = "20080101"
        
        # Read the files of returns
        self._returns = df.loc[t0:t1, symbols]
        self._no = len(self._returns.columns)

        # Ranking of returns and rescaling
        self._ranks = self._returns.rank(axis=1, numeric_only=True, method='first')
        self._ranks = 2*self._ranks.applymap(np.int) - (self._no+1)

        self._curr = [_c[4:] for _c in self._ranks.columns]
        self._ranks.columns = self._curr
        
        self._kld_df = None
        
        self._bincounts = self._ranks.apply(pd.Series.value_counts, args=(True,True,True,))

        
    # Routine to calculate the Kullback-Leibler divergence. 
    # Requires scipy.stats entropy routine.
    def _KLD(self, P, Q):
        p = np.array(P)
        q = np.array(Q)
        if len(p.shape) > 1:
            p = p.flatten()
            q = q.flatten()
        return entr(p, q)


    # Calculate Kullback-Leibler divergence between empirical distribution and the 
    # MaxEnt distribution from the marginales.
    # If we have a ranking, then rank_i != rank_j needs to be enforced.
    # Refer to mathematicaMaxEnt for this. 
    # Otherwise, np.outer suffices.
    def _S_KLD(self, X, Y, maxent_func=mathematicaMaxEnt, t0=None, t1=None):
        x = X.loc[t0:t1]
        y = Y.loc[t0:t1]

        x_bins = x.value_counts(True, True, True).sort_index().values.tolist()
        y_bins = y.value_counts(True, True, True).sort_index().values.tolist()

        maxent_distr = maxent_func(x_bins, y_bins) 
        emp_distr = pd.crosstab(x, y, normalize=True).as_matrix()
        
        return self._KLD(emp_distr, maxent_distr)
    
    
    # Calculate S_KLD for all currency pairs and return the resulting matrix.
    def _S_KLD_matrix(self, df, no, lag):
        curr = self._curr
        # If no lag, then calculate for i, j, and i!=j
        # Else, calculate all i, j pairs, but use symmetry.
        # To do that, create matrix where all pairs to be calculated are np.nan
        if lag > 0:
            kld_df = pd.DataFrame(data=np.empty((no,no,)).fill(False),
                                  index=[c+ "_predictor" for c in curr], columns=curr)
            maxent_func = np.outer
        else:
            kld_df = pd.DataFrame(data=np.identity(no),
                                  index=curr, columns=curr).replace(0, False) - 1
            maxent_func = mathematicaMaxEnt
        
        for i in range(no):
            for j in range(no):
                if kld_df.iloc[i,j]:
                    df_i, df_j = self._predictor(df.iloc[:, [i,j]], lag) 
                    kld_df.iloc[i,j] = rescaleRange(self._S_KLD(df_i, df_j, maxent_func))
                else:
                    pass
                
                if lag == 0:
                    kld_df.iloc[j,i] = kld_df.iloc[i,j]
        
        return kld_df

    
    # Instead of considering concurrent rankings, we can study lagged relationships.
    # This routine shifts one time series and makes sure that only true 10-minute intervals
    # are being considered in analysis.
    # If lag 0, funtion only returns columns separately.
    def _predictor(self, df, lag):
        if lag > 0:
            c1, c2 = df.columns
            df = pd.merge(df.iloc[:, 0].shift(lag).to_frame(c1 + "_predictor"),
                          df.iloc[:, 1].to_frame(c2), 
                          left_index=True, right_index=True)
            df = df.dropna().applymap(np.int)
            
        return df.iloc[:, 0], df.iloc[:, 1]

    
    # Time slicing according to quarterly analysis, annual analysis
    # or for whole time period
    # To-do: make more intuitive and replace all the if-else.
    def _timeslicer(self, year, q):
        # Allow to evaluate quarterly. 
        # If no quarter is specified, select whole year
            
        if year != None:
            y1, y2 = str(year), str(year)    
        else:
            y1, y2 = str(self._ranks.index[0].year), str(self._ranks.index[-1].year)
            q = None
            
        if q != None:
            q0, q1 = '{:02d}'.format(3*q-2), '{:02d}'.format(3*q)
        else:
            q0, q1 = "01", "12"
            
        
        loc0 = pd.to_datetime(y1 + "-" + q0 + "-01")
        loc1 = pd.to_datetime(y2 + "-" + q1) + MonthEnd(1)
        
        return loc0, loc1
      
    
    def get_bincounts(self):
        return self._bincounts
    
    
    def get_curr(self):
        return self._curr
    
        
    def get_kld(self):
        return self._kld
    
    
    def get_ranks(self):
        return self._ranks
    
    
    def get_ranks_shifted(self, lag, dropna=True):
        # Shift ranks by lag and return as new dataframe
        df = self.get_ranks().resample("10T").last().diff(lag)
        if dropna:
            df = df.dropna()
        return df
    
    
    def get_returns(self):
        return self._returns
    
    
    def make_kld(self, year=None, q=None, lag=0, export=True):
        # Number of currencies
        no = self._no
        
        loc0, loc1 = self._timeslicer(year, q)
    
        df = self._ranks.loc[loc0:loc1]
                
        # Calculate Kullback-Leibler divergence for all pairs of empirical and theoretical 
        # bivariate distributions and rescale to [0,1].
        kld_df = self._S_KLD_matrix(df, no, lag)
        
        # After calculating one triangle of matrix, mirror along diagonal and 
        # set diagonal to np.nan
        kld_df = kld_df.replace(0, np.nan)
        
        if export == True:
            kld_df.to_csv(stringCSV("KLD", no, year, q, "lag_"+str(lag)), index=True)
        else:
            pass
        
        self._kld = kld_df
        

In [715]:
# Parent class to download data

class macroData(object):
    def __init__(self, cset=None, t0=None, t1=None):
        # Variables for API url and access token
        self._url = None
        self._access_token = None
        
        self._cset = cset
        
        # Default state date: January 2008
        # Default end date: now
        # Format as strings as well as datetime
        if t0 == None:
            t0 = "20080101"
        self._t0 = pd.to_datetime(t0, format="%Y%m%d")
        
        if t1 == None:
            self._t1 = pd.to_datetime(time(), unit="s")
        else:
            self._t1 = pd.to_datetime(t1, format="%Y%m%d")
        
        self._st0 = str(self._t0)[0:10]
        self._st1 = str(self._t1)[0:10]
           
            
        # Open lookup table for conversions between different sources
        # Throw error if file can't be opened and read properly.
        try:
            self._lookup_df = pd.read_csv("iso_codes.csv", index_col=0).dropna()
            self._lookup_df = self._lookup_df.rename(columns={"pt3ISO":"ISO"})
            self._iso_dict = self._lookup_df.dropna().set_index("ISO").to_dict()
            self._tb_dict = self._lookup_df.dropna().set_index("FX").to_dict()["TB"]
        except IOError as e:
            print "Failing to open ISO code file.", str(e)
    
    
    # Take numeric dataframe with time as index.
    # Convert index to datetime and data to float.
    def _format_float_df(self, df):
        df.index = pd.to_datetime(df.index)
        df = df.applymap(np.float)
        
        return df
    
        
    # Routine to request data from API
    def _request(self, **params):
        # Allow for endpoint in API
        if "endpoint" in params:
            url =  "%s/%s" % (self._url, params["endpoint"])
        else:
            url = self._url
        
        try:
            response = requests.get(url, params=params)
        except requests.RequestException as e:
            print (str(e))
            
        content = response.content.decode('utf-8')
        try:
            content = json.loads(content)
        except ValueError as e:
            print "Content from API could not be decoded", str(e)
            print "Returning content as is."
        
        return content  
    
    
    # Classification of currencies as reserve or commodity
    def currency_class(self):
        c_com_dict = {'AUD': 1, 'CAD': 1, 'CHF': 0, 'EUR': 0, 'GBP': 0, 'JPY': 0, 
                      'MXN': 0, 'NOK': 1, 'NZD': 0, 'SEK': 0, 'USD': 0, 'ZAR': 1}
        c_res_dict = {'AUD': 0, 'CAD': 0, 'CHF': 1, 'EUR': 1, 'GBP': 1, 'JPY': 1, 
                      'MXN': 0, 'NOK': 0, 'NZD': 0, 'SEK': 0, 'USD': 1, 'ZAR': 0}
        
        return pd.concat([
            pd.DataFrame.from_dict(c_com_dict, orient="index").rename(columns={0:"Comm_Curr"}),
            pd.DataFrame.from_dict(c_res_dict, orient="index").rename(columns={0:"Reserve_Curr"})],
            axis=1)
            
        
    # Classification of currencies according to continent / timezone.
    def same_continent(self):
        curr = self._cset
        cont_dict = self._lookup_df.loc[:, ["FX", "Cont"]].set_index("FX").to_dict()["Cont"]
        
        continent_df = pd.DataFrame(index=curr, columns=curr)
        continent_df = continent_df.unstack().reset_index().drop(0, axis=1)
        continent_df = pd.concat([continent_df.rename(columns={"level_0":"country_1", "level_1":"country_2"}),
                                  continent_df.rename(columns={"level_0":"continent_1", "level_1":"continent_2"})], 
                                 axis=1)
        continent_df = continent_df.loc[continent_df["continent_1"] != continent_df["continent_2"]]
        continent_df = continent_df.set_index(["country_1", "country_2"])
        continent_df.loc[:, "continent_1"] = continent_df.loc[:, "continent_1"].map(cont_dict)
        continent_df.loc[:, "continent_2"] = continent_df.loc[:, "continent_2"].map(cont_dict)
        continent_df.loc[:, "same_cont"] = (continent_df.loc[:, "continent_1"]==continent_df.loc[:, "continent_2"])
        continent_df.loc[:, "same_cont"] = continent_df.loc[:, "same_cont"].replace({True:1, False:0})
        
        return continent_df.loc[:, "same_cont"]
    
    # Read file with access token.
    # To do: error handling
    def set_token(self, src):
        if src != None:
            with open(src + "_Access_Token.txt", "r") as f:
                self._access_token = f.readline()

            
    def set_url(self, url):
        self._url = url

In [716]:
# Child class to download financial data from stocks etc.
class financialData(macroData):
    def __init__(self, **kwargs):
        super(financialData, self).__init__(**kwargs)
        
        # Fill correlation matrix upon initialization
        self._corr_matrix = self._populate_corr_matrix()    
            
            
    # Calculate correlations for each year that is in 
    # the specified range upon initialization
    def _populate_corr_matrix(self):
        ix_dict = self._lookup_df.loc[:, ["FX", "IX"]].set_index("FX")
        ix_dict = ix_dict.to_dict()["IX"]
        
        df = pd.concat([self.get_stock(symbol=ix_dict[c], 
                                       resample=None).rename(columns={ix_dict[c]:c})
                        for c in self._cset], axis=1)
        df.index = pd.to_datetime(df.index)
        df.loc[:, "Year"] = df.index.year
        corr_df = df.groupby("Year").corr()
        
        corr_matrix = {}
        for y in df.loc[:, "Year"].values:
            corr_matrix[y] = corr_df.query("Year == " + str(y)).loc[y]
            
        return corr_matrix
    
    # Return correlation matrix for a given year
    def get_corr_matrix(self, year=None):
        return self._corr_matrix.get(year, "Enter valid year")
    
    
    # Return time series of financial instrument identified by
    # symbol and source like iex, morningstar, NASDAQ...
    # By default, resample quarterly and return last.
    def get_stock(self, symbol=None, source=None, 
                  start=None, end=None, resample="Q"):
        
        # Default value handling.
        if symbol == None:
            symbol = "SPY"
        if source == None:
            source = "morningstar"
        if start == None:
            start = self._st0
        if end == None:
            end = self._st1
            
        df = data.DataReader(symbol, source, start, end)
        
        df = df.reset_index()
        df = df.loc[df["Volume"] > 0, ["Date", "Close"]].rename(columns={"Close":symbol})
        df.loc[:, "Date"] = pd.to_datetime(df.loc[:, "Date"])
        df = df.set_index("Date")
        
        if resample == None:
            return df
        else:
            return df.resample(resample).last()
        
        
    # Return oil prices provided by eia.gov
    # By default, resample quarterly and return last.
    def get_oil(self, start=None, end=None, resample="Q"):
        
        if start == None:
            start = self._st0
        if end == None:
            end = self._st1
            
        oil_df = pd.read_excel("http://www.eia.gov/dnav/pet/hist_xls/RCLC1d.xls", 
                               sheet_name=1, header=2, index_col=0, parse_dates=True)
        oil_df.columns = ["Oil_Price"]
        
        return oil_df.loc[start:end].resample(resample).last()

In [717]:
class tradeData(macroData):
    def __init__(self, **kwargs):
        super(tradeData, self).__init__(**kwargs)
        self.set_url("http://comtrade.un.org/api/get")
        self.set_token(None)
        
        self._trade_matrix = {}
        self._check_local_files(self._t0.year, self._t1.year)
        
        
    def _check_local_files(self, y0, y1):
        missing_yr = []
        for yr in range(y0, y1+1):
            try:
                self._trade_matrix[yr] = pd.read_csv("data/trade_matrix_"+str(yr)+".csv",
                                                     index_col=0)
            except:
                missing_yr.append(str(yr))
        
        if len(missing_yr) == 0:
            print "Trade matrices complete."
        else:
            print "Trade matrices missing: ", ", ".join(missing_yr)
            
            
    def _year_string(self, y0, y1):
        return [",".join(map(str, year_range)) 
                for year_range 
                in np.array_split(np.arange(y0,y1+1), (y1-y0)/3 + 1)]
    
    
    def _timing_wrapper(func):
        from time import sleep
        
        def wrapper(*args, **kwargs):
            sleep()
            return func(*args, **kwargs)
        return wrapper
    
    
    @_timing_wrapper
    def _request(self, **params):
        content = super(tradeData, self)._request(**params)
        return content
    
    
    def _pull_data(self, **params):
        params["fmt"] = "json"
        params["cc"]  = "TOTAL"
        
        content = self._request(**params)
        assert type(content) == dict, "Data is not dict\n" + content
        content = content["dataset"]
        
        df = pd.DataFrame.from_dict(content)
        if len(df) > 0:
            df = df.loc[(df["rgDesc"] == "Import") | (df["rgDesc"] == "Export")]
            df = df.loc[:, ["yr", "rgDesc", "rt3ISO", "pt3ISO", "TradeValue"]]  
            df.loc[:, "yr"] = pd.to_datetime(df.loc[:, "yr"], format="%Y")
            df.loc[:, "TradeValue"] = df.loc[:, "TradeValue"].apply(np.int)
            df = df.rename(columns={"yr": "Year", "rgDesc": "Kind",
                                    "rt3ISO": "Reporter", "pt3ISO": "Partner"})
            return df.set_index("Year")
        else:
            pass
    
    
    def get_country_trade(self, country, **params):
        y0, y1 = self._t0.year, self._t1.year
        lookup_df = self._lookup_df.dropna()
        lookup_df = lookup_df.loc[lookup_df["FX"].isin(self._cset)]

        un_codes = lookup_df.loc[lookup_df["FX"] == country, "UN"].values
        
        dl_years = self._year_string(y0, y1)
        
        df = pd.concat([
            pd.concat([self._pull_data(r=str(reporter), p="All", ps=y) 
                       for y in dl_years])
            for reporter in un_codes])
        
        df.loc[:, "Reporter"] = df.loc[:, "Reporter"].map(self._iso_dict["FX"])
        df.loc[:, "Partner"] = df.loc[:, "Partner"].map(self._iso_dict["FX"])
        df = df.loc[df["Partner"].isin(self._cset)].reset_index()
        df = df.groupby(["Year", "Reporter", 
                         "Kind", "Partner"]).sum().drop(country, level=3)
        
        return df
    
    
    def get_trade_matrices(self, year=None):
        return self._trade_matrix.get(year, "Enter valid year")

In [718]:
class econoData(macroData):
    def __init__(self, **kwargs):
        super(econoData, self).__init__(**kwargs)
        self.set_url("https://api.stlouisfed.org/fred")
        self.set_token("FRED")

        
    def _add_keywords(self, **params):
        params["endpoint"] = "series/observations"
        params["file_type"] = "json"
        params["api_key"] = self._access_token
        
        if "start" in params:
            time = params.pop("start")
            params["realtime_start"] = time
        if "end" in params:
            time = params.pop("end")
            params["realtime_end"] = time
            
        return params
        
    
    def _pull_data(self, **params):
        start = params.get("start", self._st0)
        end = params.get("end", "9999-12-31")
        
        params = self._add_keywords(**params)
        
        content = self._request(**params)
        content = content["observations"]
        
        df = pd.DataFrame.from_dict(content)
        
        assert len(df) > 0, "No data for " + params["series"].upper()
        df = df.loc[df["date"] >= start, ["date", "value"]].set_index("date")
        
        return self._format_float_df(df)
    
    
    def get_series(self, series=None, country=None, **params):
        def api_str(series, country):
            series_dict = {"GDP": ("MKTGDP", "A646NWDB"),
                           "10YR": ("IRLTLT01", "A156N"), 
                           "UNEMP": ("LRHUTTTT", "A156N")}
            return series_dict[series][0] + country + series_dict[series][1]
            
        if series == None:
            series = "10YR"
        if country == None:
            country = "US"
        
        series, country = series.upper(), country.upper()
        
        params["series_id"] = api_str(series, country)
            
        df = self._pull_data(**params)
        
        return df.rename(columns={"value":country+"_"+series})

In [186]:
ed = econoData(cset=curr, t0="20120101", t1="20171231")
cs = currencySet(df, symbols=symbols, t0="20120101", t1="20171231")
fd = financialData(cset=curr, t0="20120101", t1="20171231")
ct = tradeData(cset=curr, t0="20120101", t1="20171231")

Trade matrices complete.


In [30]:
kld_dict = {}
for y in range(2012,2018):
    cs.make_kld(export=True, year=y)
    kld_dict[y] = cs.get_kld()

In [832]:
def plot_kld(year, export=False):
    fig, ax = plt.subplots(figsize=(6.5,5))
    plot_df = kld_dict[year]
    sns.heatmap(plot_df, cmap="Reds", vmax=0.4, square=True, ax=ax)
    ax.set_xlabel("Currency", fontdict={"size":18})
    ax.set_ylabel("Currency", fontdict={"size":18})

    plt.setp(ax.get_xticklabels(), rotation='vertical', fontsize=12);
    plt.setp(ax.get_yticklabels(), rotation='horizontal', fontsize=12);
    ax.set_title("KL-Divergence in " + str(year),fontsize=20);

    plt.tight_layout();
    if export:
        fig.savefig("fig/KLD_annual_"+ str(year)+ ".png", bbox_inches="tight");

In [833]:
for y in range(2012,2018):
    plot_kld(y, True)

In [ ]:
# Interest rates / Fisher effect
# Purchasing power
# 10 year yields
# GDP
# Unemployment